In [1]:
# @title Setup & Imports
from typing import List, Dict, Tuple, Optional, Union, Literal, Annotated, TypedDict, Callable, Any, TypeVar, Generic
from pydantic import BaseModel, Field, ValidationError

## 🔹 Basic Types in Typing

These are used heavily in Pydantic models to define expected structure of data:
- `List[]` → a list of items of a certain type
- `Dict[key_type, value_type]` → dictionary
- `Tuple[]` → fixed-size group of elements
- `Optional[]` → value can be of type OR `None`


In [26]:
# List
my_list: list = [1, 2, 3]
another_list: list[int] = [10, 20, 30] # Type hint for clarity

# Dictionary
my_dict: dict = {"a": 1, "b": 2}
another_dict: dict[str, int] = {"c": 3, "d": 4} # Type hint for clarity

# Tuple
my_tuple: tuple = (1, "hello")
another_tuple: tuple[int, str] = (5, "world") # Type hint for clarity
fixed_size_tuple: tuple[int, str, bool] = (10, "test", True)

# Optional (using Union with None)
value_or_none: Union[int, None] = 10
another_value_or_none: Union[str, None] = None

def process_value(value: Union[int, None]):
    if value is not None:
        print(f"Processing integer: {value}")
    else:
        print("No value provided")

process_value(value_or_none)
process_value(another_value_or_none)

# Note: Without Pydantic, these are just type hints. They do not enforce validation
# at runtime. You would need to add explicit checks for type and structure.

Processing integer: 10
No value provided


In [3]:
class User(BaseModel):
    name: str
    tags: List[str]
    metadata: Dict[str, int]
    address: Optional[str]
    coordinates: Tuple[float, float]

example = {
    "name": "Shreyash",
    "tags": ["ai", "developer"],
    "metadata": {"age": 25},
    "coordinates": [78.4, 21.2],
    "address": None # Add the missing 'address' key with None value
}

user = User(**example)
print(user)

name='Shreyash' tags=['ai', 'developer'] metadata={'age': 25} address=None coordinates=(78.4, 21.2)


## 🔹 Union - Accept Multiple Possible Types
Use case: Sometimes values can come in as different formats (e.g., str OR int)

In [25]:
class Payment(BaseModel):
    amount: Union[int, float, str]  # all acceptable

payment = Payment(amount="1500")
print(payment.amount)
payment = Payment(amount="Fourteen Hundred")
print(payment.amount)
payment = Payment(amount=1300.0)
print(payment.amount)

1500
Fourteen Hundred
1300.0


## 🔹 Literal and Annotated
- `Literal`: restricts allowed values
- `Annotated`: adds metadata (like `Field()` constraints) to the type

In [5]:
class Order(BaseModel):
    status: Literal["pending", "completed", "cancelled"]
    discount: Annotated[int, Field(gt=0, lt=100)]

order = Order(status="completed", discount=20)
print(order)

status='completed' discount=20


## 🔹 TypedDict — define structure of a dict without converting it into class
Useful when you don't want to use a full Pydantic model but still want structured typing.

In [23]:
class AddressDict(TypedDict):
    street: str
    city: str
    pincode: int

def print_address(addr: AddressDict):
    print(f"{addr['street']}, {addr['city']} - {addr['pincode']}")

print_address({"street": "MG Road", "city": "Pune", "pincode": "E411001"})#sending 3 sting value instead of 2 but still running

MG Road, Pune - E411001


## 🔹 Callable — Specify input and return types of functions

In [7]:
def apply_discount(price: float, tax: float) -> float:
    return price * (1 - tax)

def call_discount(fn: Callable[[float, float], float]) -> float:
    return fn(1000, 0.2)

print(call_discount(apply_discount))

800.0


## 🔹 Any, TypeVar, Generic — for reusable & flexible type-safe code
- `Any`: disables type-checking
- `TypeVar`: like a template type (generic programming)
- `Generic`: build reusable containers


In [17]:
T = TypeVar('T') # Define a type variable 'T'

class Box(Generic[T]):
    """A simple generic box that can hold an item of any type T."""
    def __init__(self, item: T):
        self.item = item

    def get_item(self) -> T:
        return self.item

# Example using Any:
# Use Any when the type is truly unknown or dynamic, but you lose type safety.
def process_anything(data: Any):
    """Processes data of any type."""
    print(f"Processing data of type: {type(data)}")
    # You can perform operations that might fail if the type isn't what you expect
    # For example, trying to access a method that only exists on certain types
    # try:
    #     print(data.upper()) # This would fail for int, list, etc.
    # except AttributeError:
    #     print("Cannot apply .upper()")

print("\n--- Using Any ---")
process_anything("hello")
process_anything(123)
process_anything([1, 2, 3])

# Example using TypeVar and Generic:
# Use Generic with TypeVar to create type-safe containers or functions
# that work with different types while preserving type information.

print("\n--- Using Generic and TypeVar ---")
int_box = Box[int](10) # The box is explicitly typed to hold integers
print(f"Integer box item: {int_box.get_item()} (Type: {type(int_box.get_item())})")

string_box = Box[str]("Colab") # The box is explicitly typed to hold strings
print(f"String box item: {string_box.get_item()} (Type: {type(string_box.get_item())})")

# Type checking will help here:
# int_box_error = Box[int]("hello") # This would be flagged by a type checker

# Another example using a generic function
U = TypeVar('U')

def first_element(items: List[U]) -> Optional[U]:
    """Returns the first element of a list, preserving its type."""
    if items:
        return items[0]
    return None

print(f"First int: {first_element([1, 2, 3])} (Type: {type(first_element([1, 2, 3]))})")
print(f"First str: {first_element(['a', 'b', 'c'])} (Type: {type(first_element(['a', 'b', 'c']))})")
print(f"First None: {first_element([])} (Type: {type(first_element([]))})")


--- Using Any ---
Processing data of type: <class 'str'>
Processing data of type: <class 'int'>
Processing data of type: <class 'list'>

--- Using Generic and TypeVar ---
Integer box item: 10 (Type: <class 'int'>)
String box item: Colab (Type: <class 'str'>)
First int: 1 (Type: <class 'int'>)
First str: a (Type: <class 'str'>)
First None: None (Type: <class 'NoneType'>)


In [16]:
T = TypeVar('T') # Define a TypeVar T

def first_element(items: List[T]) -> T:
    """Returns the first element of a list, inferring the type."""
    if not items:
        raise ValueError("List cannot be empty")
    return items[0]

# Example usage with different list types
int_list: List[int] = [1, 2, 3]
first_int: int = first_element(int_list)
print(f"First element of integer list: {first_int}")

str_list: List[str] = ["apple", "banana", "cherry"]
first_str: str = first_element(str_list)
print(f"First element of string list: {first_str}")

# You could also use TypeVar with Generic classes
T_Container = TypeVar('T_Container')

class Box(Generic[T_Container]):
    def __init__(self, item: T_Container):
        self._item = item

    def get_item(self) -> T_Container:
        return self._item

int_box: Box[int] = Box(10)
item_from_int_box: int = int_box.get_item()
print(f"Item from integer box: {item_from_int_box}")

str_box: Box[str] = Box("hello")
item_from_str_box: str = str_box.get_item()
print(f"Item from string box: {item_from_str_box}")

First element of integer list: 1
First element of string list: apple
Item from integer box: 10
Item from string box: hello


In [9]:
T = TypeVar("T")

class Wrapper(Generic[T]):
    def __init__(self, value: T):
        self.value = value

# Create an instance of Wrapper for int_wrap, providing a value
int_wrap = Wrapper[int](123)
str_wrap = Wrapper[str]("hello")

print(int_wrap.value)
print(str_wrap.value)

123
hello


## ✅ LangGraph AgentState Example using TypedDict + Annotated

In [11]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.1 MB/s eta 0:00:00


In [12]:
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages: Annotated[List[dict], add_messages]
    user_input: str
    summary: Optional[str]

## ✅ LangChain tool decorator needs precise Callable typing

In [14]:
from langchain_core.tools import tool

@tool
def calculate_total(price: float, tax: float) -> float:
    "Perform Price Calculations"
    return price + (price * tax)

print(calculate_total.run({"price": 100, "tax": 0.18}))

118.0


In [15]:
class ToolInput(BaseModel):
    product: Annotated[str, Field(description="Product name")]
    quantity: Annotated[int, Field(gt=0, description="How many items")]

@tool
def generate_invoice(product: str, quantity: int) -> str:
    "Help us Generate Invoice"
    return f"Invoice for {quantity} units of {product}"

## 🔄 Flow of Typing Usage in LangChain Agents

1. **Define State with `TypedDict`** → shared memory for graph nodes
2. **Use `Annotated` for attaching metadata (e.g. LangGraph's `add_messages`)**
3. **Use `@tool` functions with `Callable` typing** → allows external tools to run
4. **Pydantic helps validate and constrain input/output for each node**



## ✅ You’ve Learned:
- 🔹 List, Dict, Tuple, Optional
- 🔹 Union, Literal, Annotated
- 🔹 TypedDict (structure without class)
- 🔹 Callable functions
- 🔹 TypeVar and Generic
- 🔹 Pydantic models with type constraints
- 🔹 LangChain tools & state with Typing

Next → Use these concepts to build:
1. Typed LangGraph pipelines
2. Pydantic-based dynamic tool agents
3. LangChain Runnable trees with validated inputs


In [ ]:
#Act as a Python Expert who have worked on various advance concepts of Typing Libraries in Python with various Real life like use cases. now I am learning langchain and langgraph and in that I have seen Typing lybraries been used in various use case time to time, so I though I should Learn Typing for myself. Now as someone who never know anything about typing. help me understand and learn best practices of typing library used on regular bases, and also help me learn how its useful while using pydantic, things I need to know in typing before learning langchain and langgraph. Dont explain every single thing in typing just focus on the components used in above mention use cases. "https://medium.com/@moraneus/exploring-the-power-of-pythons-typing-library-ff32cec44981", for teaching me all this you can help me create an google colab notebook ipynb file where you are first explaining a component, concept or practices, then explaining it through a simple program through sample code that can help me understand those component, concept or practices completely. make sure to create proper markdown and comments with flowchart(if required and can use tags and all for it) so I can understand theory and maths that works behind during that component, concept or practices, make sure to keep the explaination brief short and beginner-friendly. and only cover key topics not all of them mostly focus on Typing Libraries in Python with its best component, concept or practices and its most common usecase with pydantic. and then do the same for Typing Librarie's best component, concept or practices with respect to langchain and langgraph use case.

In [ ]:
#Act as a Python Expert who have worked on various advance concepts of Typing Lybraries in Python with various Real life like use cases. now I am learning langchain and langgraph and in that I have seen Typing lybraries been used in various use case time to time, so I though I should Learn Typing for myself. Now as someone who never know anything about typing. help me understand and learn best practices of typing library used on regular bases, and also help me learn how its usefull while using pydantic, things I need to know in typing before learning langchain and langgraph. Dont explain every single thing in typing just focus on the components used in above mention use cases. "https://medium.com/@moraneus/exploring-the-power-of-pythons-typing-library-ff32cec44981", for teaching me all this you can help me create an google colab notebook ipynb file where you are first explaining a component, concept or practices, then explaining it through a simple program through sample code that can help me understand those component, concept or practices compleatly. make sure to create proper markdown and comments with flowchart(if required and can use tags and all for it) so I can understand theory and maths that works behind during that component, concept or practices like beginner-friendly.